**Azalea Yunus x Benji Andrews**

Fall 2020

CS 343: Neural Networks

Project 2: Multi-layer Perceptrons

**Submission reminders:**

- Submit rubric on Google Classroom (one per team)
- Submit one .zip file per team on Google Classroom. Includes:
    - All .ipynb notebook files
    - All .py code files
    - Data files under 10 MB
- Did you answer all 9 questions?

In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

# for obtaining the STL-dataset
import load_stl10_dataset

# for preprocessing dataset
import preprocess_data

# Set the color style so that Professor Layton can see your plots
plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
# Make the font size larger
plt.rcParams.update({'font.size': 20})

# Turn off scientific notation when printing
np.set_printoptions(suppress=True, precision=3)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

def plot_cross_entropy_loss(loss_history):
    plt.plot(loss_history)
    plt.xlabel('Training iteration')
    plt.ylabel('loss (cross-entropy)')
    plt.show()

## Load in data

### a. STL-10

Run your function to load in the preprocessed STL-10 data in the following split:

- 3500 training samples
- 500 test samples
- 500 validation samples
- 500 samples for development

In [2]:
random.seed(0)
x_train, y_train, x_test, y_test, x_val, y_val, x_dev, y_dev = preprocess_data.load_stl10()
test_imgs = x_dev[-15:,:]
test_labels = y_dev[-15:]
print("shape test imgs", test_imgs.shape)
print("shape test labels", test_labels.shape)

Images are: (5000, 96, 96, 3)
Labels are: (5000,)
Resizing 5000 images to 32x32...Done!
Saving Numpy arrays the images and labels to ./numpy...Done!
shape test imgs (15, 3072)
shape test labels (15,)


### b. Circle in a square

The circle in a square (CIS) dataset is a simple binary classification dataset that is useful for debugging and visualizing what your MLP is learning. Points with (x, y) coordinates inside a circle have class value of 1, points with coordinates outside the circle have class value of 0. Training on the CIS dataset allows us to answer the question: can the MLP discriminate whether a test point falls inside or outside the circle?

#### Todo

- Download the CIS dataset then run the cell below to load in the CIS train (`cis_train.dat`) and test (`cis_test.dat`) sets as numpy arrays.
- Below, make a scatterplot showing the test set data. Color-code samples based on their class. If everything goes well, you should see a...solid, filled in circle inside unit square :)
    - Make the aspect ratio of your x, y plotting axes equal, otherwiwse you might see an ellipse!

In case you're curious about the data format:
- Like usual, each row is a different sample.
- The x-coordinate feature is the 1st column
- The y-coordinate feature is the 2nd column
- The class label (0 or 1) is in the third column.


In [3]:
val_size = 20

cis_train_path = os.path.join('data', 'cis', 'cis_train.dat')
cis_test_path = os.path.join('data', 'cis', 'cis_test.dat')

cis_train_all = np.loadtxt(cis_train_path, delimiter='\t')

# shuffle the data
s_inds = np.arange(len(cis_train_all))
np.random.seed(0)
np.random.shuffle(s_inds)

cis_train_all = cis_train_all[s_inds]

cis_train_x = cis_train_all[:, :2]
cis_train_y = cis_train_all[:, 2].astype(int)

cis_val_x = cis_train_x[:val_size]
cis_train_x = cis_train_x[val_size:]
cis_val_y = cis_train_y[:val_size]
cis_train_y = cis_train_y[val_size:]

cis_test_all = np.loadtxt(cis_test_path, delimiter='\t')
cis_test_x = cis_test_all[:, :2]
cis_test_y = cis_test_all[:, 2].astype(int)

print ('CIS Train data shape: ', cis_train_x.shape)
print ('CIS Train labels shape: ', cis_train_y.shape)
print ('CIS Validation data shape: ', cis_val_x.shape)
print ('CIS Validation labels shape: ', cis_val_y.shape)
print ('CIS Test data shape: ', cis_test_x.shape)
print ('CIS Test labels shape: ', cis_test_y.shape)

OSError: data/cis/cis_train.dat not found.

In [ ]:
# scatter plot here
plt.scatter(cis_test_all[:,0], cis_test_all[:,1], c=cis_test_all[:,2])
plt.axis('equal')
plt.title("CIS Test Data")
plt.show()

## Task 3: Implement Multilayer Perceptron (MLP) with softmax activation and cross-entropy loss

Now that we've tested the softmax activation function and cross-entropy loss functions in a single-layer net, let's implement the MLP version.

Much of your work on the single layer net will carry over, so go ahead and copy-paste and modify as needed!

The structure of our MLP will be:

```
Input layer (X units) ->
Hidden layer (Y units) with Rectified Linear activation (ReLu) ->
Output layer (Z units) with softmax activation
```

### 3a. Implement the following functions in `mlp.py`

- `initialize_wts`
- `accuracy`
- `one_hot`
- `predict`
- `forward`
- `backward`
- `fit`

### 3b. Test key functions with randomly generated data

In [ ]:
from mlp import MLP

In [ ]:
# Create a dummy net for debugging
num_inputs = 3
num_features = 6
num_hidden_units = 7
num_classes = 5

net = MLP(num_features, num_hidden_units, num_classes)

In [ ]:
# Generate random data and classes
np.random.seed(0)
test_x = np.random.normal(loc=0, scale=100, size=(num_inputs, num_features))
test_y = np.random.uniform(low=0, high=num_classes-1, size=(num_inputs,))
test_y = test_y.astype(int)
print(f'Test input shape: {test_x.shape}')
print(f'Test class vector shape: {test_y.shape}')

#### Test `initialize_wts`

In [ ]:
net.initialize_wts(M=num_features, H=num_hidden_units, C=num_classes, std=0.01)
print(f'y wt shape is {net.y_wts.shape} and should be (6, 7)')
print(f'y bias shape is {net.y_b.shape} and should be (7,)')
print(f'z wt shape is {net.z_wts.shape} and should be (7, 5)')
print(f'z bias shape is {net.z_b.shape} and should be (5,)')

print(f'1st few y wts are\n{net.y_wts[:,0]}\nand should be\n[ 0.018 -0.002  0.004  0.007  0.015  0.002]')
print(f'y bias is\n{net.y_b}\nand should be\n[-0.017  0.02  -0.005 -0.004 -0.013  0.008 -0.016]')
print(f'1st few z wts are\n{net.z_wts[:,0]}\nand should be\n[-0.002 -0.    -0.004  0.002  0.001  0.004  0.001]')
print(f'z bias is\n{net.z_b}\nand should be\n[ 0.015  0.019  0.012 -0.002 -0.011]')

#### Test the `predict` method

In [ ]:
test_y_pred = net.predict(test_x)
print(f'Predicted classes are {test_y_pred} and should be [3 0 0]')

#### Test the `forward` method focusing on`ReLU`(net act of hidden layer `y`)

In [ ]:
_,y_net_act_test,_,_,_ = net.forward(test_x, test_y)

correct_y_act = np.array([[7.643, 4.49 , 0.799, 9.977, 0.   , 0.   , 0.   ],
       [2.353, 2.737, 2.175, 2.547, 0.345, 0.   , 0.   ],
       [3.98 , 2.691, 1.19 , 3.029, 0.   , 0.   , 0.   ]])

print(f'Your y activation is\n{y_net_act_test}')
print(f'The correct y activation (ReLU) is\n{correct_y_act}')

#### Test the `forward` method

In [ ]:
_,_,_,probs,_ = net.forward(test_x, test_y)

correct_probs = np.array([[0.219, 0.2  , 0.191, 0.219, 0.171],
       [0.208, 0.204, 0.201, 0.205, 0.183],
       [0.208, 0.202, 0.202, 0.205, 0.183]])

print(f'Your z activation (class probabilities) is\n{probs}')
print(f'The correct z activation (class probabilities) is\n{correct_probs}')
print(f'The sums across rows (for each data sample) are {np.sum(probs, axis=1)}.')
print(f'  You should know what should be :)')

#### Test the `forward` method, focusing on loss

In [ ]:
y_in, y_act ,z_in, z_act, loss = net.forward(test_x, test_y)
correct_loss = 1.564402690536365

print(f'Your average loss is\n{loss}')
print(f'The correct average loss is approx\n{correct_loss}')

#### Test the `forward` method, focusing on regularization

In [ ]:
y_in, y_act ,z_in, z_act, loss = net.forward(test_x, test_y, reg=1000)
correct_loss = 5.257207314928798

print(f'Your regularized average loss is\n{loss}')
print(f'The correct regularized average loss is approx\n{correct_loss}')

#### Test the `backward` method

In [ ]:
y_in, y_act ,z_in, z_act, loss = net.forward(test_x, test_y, reg=0.5)
grads = net.backward(test_x, test_y, y_in, y_act ,z_in, z_act, reg=0.5)

print('Your gradient for y_wts is\n', grads[0])
print('Your gradient for y_b is\n', grads[1])
print('Your gradient for z_wts is\n', grads[2])
print('Your gradient for z_b is\n', grads[3])

The correct gradients are:

`
Your gradient for y_wts is
 [[-0.476  0.057 -0.458 -0.115  0.03  -0.005  0.005]
 [-0.002  0.014 -0.046 -0.162  0.004  0.004  0.001]
 [-0.088  0.038 -0.166 -0.325 -0.001 -0.004 -0.013]
 [-0.331  0.067 -0.398 -0.332  0.001  0.    -0.001]
 [-0.318  0.089 -0.465 -0.615 -0.001 -0.01  -0.002]
 [-0.315 -0.036 -0.036  0.806  0.029 -0.005 -0.007]]
Your gradient for y_b is
 [-0.005  0.    -0.004 -0.     0.     0.     0.   ]
Your gradient for z_wts is
 [[-2.879  0.933  0.131  0.987  0.816]
 [-1.69   0.669 -0.261  0.699  0.584]
 [-0.374  0.278 -0.45   0.284  0.242]
 [-3.221  1.041  0.154  1.111  0.904]
 [ 0.024  0.027 -0.091  0.029  0.015]
 [ 0.002 -0.003 -0.004 -0.003 -0.002]
 [ 0.    -0.006  0.005  0.002 -0.008]]
Your gradient for z_b is
 [-0.455  0.202 -0.135  0.209  0.179]
`

#### Test loss over epoch (1 of 2). 

The below code should generate a curve that rapidly drops to 0 (there might be fluctuations and it might not be monotonic and that's ok)

Your `fit` function should show you print-outs showing:
- Loss and validation accuracy 4 times throughout training.
- 100% accuracy on validation set after around 5 epochs of training.
- You are training on 20 epochs.
- There are 20 iterations.
- There is 1 iteration per epoch.

Here is an example print-out from `fit`:

    Starting to train network...There will be 20 epochs and 20 iterations total, 1 iter/epoch.
     Completed Epoch 0/19. Training loss: 3.78. Validation accuracy: 33.33%.
     Completed Epoch 5/19. Training loss: 0.13. Validation accuracy: 100.00%.
     Completed Epoch 10/19. Training loss: 0.22. Validation accuracy: 100.00%.
     Completed Epoch 15/19. Training loss: 0.15. Validation accuracy: 100.00%.
    Finished training!


In [ ]:
net = MLP(num_features, num_hidden_units, num_classes)
loss_hist, acc_train, acc_valid = net.fit(test_x, test_y, test_x, test_y, reg=0, print_every=5, lr=0.001, mini_batch_sz=3, n_epochs=20)

print('\nLengths of each output list:')
print(f'{len(loss_hist)=}, {len(acc_train)=}, {len(acc_valid)=}')
print('Each should be 20.')

plot_cross_entropy_loss(loss_hist)

#### Test loss over epoch (2 of 2). 

The below curve should be more jagged that above, but still converge to 0 loss.

Your `fit` function should print out:
- Loss and validation accuracy 5 times throughout training.
- 100% accuracy on validation set after around 4 epochs of training.
- You are training on 10 epochs.
- There are 30 iterations.
- There are 3 iterations per epoch.


In [ ]:
net = MLP(num_features, num_hidden_units, num_classes)
loss_hist, acc_train, acc_valid = net.fit(test_x, test_y, test_x, test_y, reg=0, print_every=2, lr=0.001, mini_batch_sz=1, n_epochs=10)

print('\nLengths of each output list:')
print(f'{len(loss_hist)=}, {len(acc_train)=}, {len(acc_valid)=}')
print('The lengths should be 30, 10, 10.')

plot_cross_entropy_loss(loss_hist)

### 3c. Test MLP with Circle in Square dataset

Before you run your MLP on the STL-10 dataset, test it out on the simpler CIS dataset.

In cells below:
- Train an MLP using the CIS training and validation sets. Configure the MLP with the following non-default hyperparameters:
    - 50 hidden units
    - Learning rate of 0.5
    - Mini-batch size of 80
    - 1000 epochs
- Plot the loss over training iterations. You should see:
    - A nice drop and plateau in mini-batch training loss.
    - Accuracy on the validation set reach ~90%.
- Create a scatterplot of the MLP predictions on the CIS test set. Color-code each sample by its class. Make sure your axis aspect ratios are equal.

In [ ]:
cis_net = MLP(2, 50, 2)
loss, val, train = cis_net.fit(cis_train_x, cis_train_y, cis_val_x, cis_val_y, n_epochs=1000, lr=0.5, mini_batch_sz=80, verbose=1)
plot_cross_entropy_loss(loss)

In [ ]:
test_pred = cis_net.predict(cis_test_x)
plt.scatter(cis_test_all[:,0], cis_test_all[:,1], c=test_pred)
plt.axis('equal')
plt.title("MLP Predicted CIS Test Data")
plt.show()

**Question 5**: How do you interpret the circle-in-square scatterplot? Is the MLP doing a good job? 

**Question 6**: Play with
- number of hidden units
- number of epochs
- batch size

How does each parameter affect the results?

**Question 7**: Do you think the single-layer net (with softmax) can handle the CIS dataset? Why or why not? (You're invited to try it, maybe as an extension :)

In [ ]:
# decrease number of hidden units from 50 to 10
cis_h = MLP(2, 10, 2)
loss_cis_h, train_acc_cis_h, val_acc_cis_h = cis_h.fit(cis_train_x, cis_train_y, cis_val_x, cis_val_y, lr=0.5, mini_batch_sz=80, n_epochs=1000)
# increase number of epochs from 1000 to 2500
cis_e = MLP(2, 50, 2)
loss_cis_e, train_acc_cis_e, val_acc_cis_e = cis_e.fit(cis_train_x, cis_train_y, cis_val_x, cis_val_y, lr=0.5, mini_batch_sz=80, n_epochs=2500)
# decrease batch size from 80 to 10
cis_b = MLP(2, 50, 2)
loss_cis_b, train_acc_cis_b, val_acc_cis_b = cis_b.fit(cis_train_x, cis_train_y, cis_val_x, cis_val_y, lr=0.5, mini_batch_sz=10, n_epochs=1000)

In [ ]:
plt.plot(loss_cis_h)
plt.xlabel('Training iteration')
plt.ylabel('loss (cross-entropy)')
plt.show()

In [ ]:
plt.plot(loss_cis_e)
plt.xlabel('Training iteration')
plt.ylabel('loss (cross-entropy)')
plt.show()

In [ ]:
plt.plot(loss_cis_b)
plt.xlabel('Training iteration')
plt.ylabel('loss (cross-entropy)')
plt.show()

In [ ]:
test_pred_h = cis_h.predict(cis_test_x)
plt.scatter(cis_test_all[:,0], cis_test_all[:,1], c=test_pred_h)
plt.axis('equal')
plt.title("MLP Predicted CIS Test Data - Fewer Hidden Layers")
plt.show()

In [ ]:
test_pred_e = cis_e.predict(cis_test_x)
plt.scatter(cis_test_all[:,0], cis_test_all[:,1], c=test_pred_e)
plt.axis('equal')
plt.title("MLP Predicted CIS Test Data - More Epochs")
plt.show()

In [ ]:
test_pred_b = cis_b.predict(cis_test_x)
plt.scatter(cis_test_all[:,0], cis_test_all[:,1], c=test_pred_b)
plt.axis('equal')
plt.title("MLP Predicted CIS Test Data - Smaller Batch Size")
plt.show()

**Answer 5**: I would say that the MLP is doing a reasonably good job, since the predicted shape resembles a circle in a square like we saw earlier. However, the circle is not perfectly round and symmetrical which indicates that the MLP could be performing better.

**Answer 6**: By decreasing the number of hidden units and keeping the number of epochs and batch size the same, the prediction got worse as the circle became more angular and less round. By increasing the number of epochs, the prediction got better as the circle became more round. By decreasing the batch size, the prediction also got better and the circle became rounder.

**Answer 7**: The single-layer network probably could not handle the CIS dataset if it used a softmax function. The ReLU function that we implemented in the first net activation is a more appropriate choice than the softmax function for the CIS dataset because of how each data sample is assigned one of two values. It doesn't make much sense to solely implement the softmax function a single-layer network for this dataset because each data sample only has its designated class as its sole feature. Furthermore, a nonlinear decision boundary is a task better suited to a net using the ReLU function.

### 3d. Test on STL-10 dataset, plot performance

Train an MLP on the STL-10 training set with the following non-default hyperparameters:
- 50 hidden units
- Learning rate of 0.1
- Regularization strength of 0.001
- Mini-batch size of 500
- 100 epochs
    
Make two plots:
- Plot the training loss (like usual).
- Plot the training and validation set accuracies (2 curves in one plot — include a legend, title, axis labels, etc.).

In [ ]:
stl_net = MLP(3072, 50, 10)
loss, val, train = stl_net.fit(x_train, y_train, x_val, y_val, n_epochs=100, lr=0.1, mini_batch_sz=500, print_every=5, verbose=1, reg=0.001)
plot_cross_entropy_loss(loss)

In [ ]:
plt.plot(val)
plt.plot(train)
plt.legend(["validation set accuracy","training set accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.show()

**Question 7**: What do the above loss and training and validation accuracy curves suggest about the quality of the hyperparameters used during training?

**Answer 7:** The above figures show that the net is underfit and could use more training. This is evidenced by the somewhat linear decline in loss over iterations, and the higher validation set accuracy than training set accuracy with significant gap between the two.


### 3e. Optimize on STL-10 dataset with random search

To optimize your MLP hyperparameters on STL-10, try a **random search** rather than a grid search. This means that instead of defining preset *values* that each hyperparameter takes on, define *ranges* (min and max values).

Run your search for some $T$ iterations. On each iteration, randomly assign values to each hyperparameter within their valid ranges.

Just like grid search, print out the accuracy and parameter values every time a bout of training yields the best accuracy on the STL-10 validation set. That way, if you need to stop the search prematurely, you know the current best parameter combination.

Consider the following hyperparameters:
- learning rate
- regularization strength
- number of hidden units
- mini-batch size

**Important note:** Like usual, I am not grading based on your performance numbers. I want to see that you successfully implemented the random search to find progressively better hyperparameters on STL-10.

**Tip:** Just like with grid search, if you find a cluster of parameters that seems promising, you can revise your search to hone in on that smaller range.

In [ ]:
import time
import multiprocessing as mp
random.seed(0)

def grid_search(lr_range, batch_range, reg_range, hiddenlayer_range, num_threads=5):
    '''
    a function to perform a grid search, finding best hyperparameters to train the net with.
    lr_range/batch_range/reg_range: ndarray. length of each determines dimension of grid search.
        these are best as output of np.linspace or similar functions.
    num_threads: int
    '''
        
    tic = time.time()
    #set ranges

    #assign lists to each of the threads
    hyper_list = []
    for i in range(num_threads):
        hyper_list.append([])

    count=0
    for lr in lr_range:
        for batch in batch_range:
            for reg in reg_range:
                    for hiddenlayer in hiddenlayer_range:
                        hyper_list[count%num_threads].append([lr,batch,reg, hiddenlayer])
                        count+=1
    threadlist = []
    for i in range(num_threads):
        newT = mp.Process(target = train_thread, args = (i,hyper_list[i].copy()))
        newT.start()
        threadlist.append(newT)


    for thread in threadlist:
        thread.join()
    toc = time.time()

    print(f"time was {toc-tic}")
                     
def train_thread(net_id, queue):
    '''
    helper method grid search fn.
    net_id: int. an id for the net so you can tell the threads apart in printouts
    queue: python list of lists. each entry is a job to train the net on, format: [lr, batch size, reg, hiddenlayers]
    '''
    best_acc = 0
    count = 0
    for hyper_combo in queue:
        net = MLP(3072, hyper_combo[3], 10)
        loss_history = net.fit(x_train, y_train, x_val, y_val, print_every=100, n_epochs=100, verbose=1, lr=hyper_combo[0], mini_batch_sz=hyper_combo[1], reg=hyper_combo[2])
        val_acc = net.accuracy(y_val, net.predict(x_val))
          
        if val_acc > best_acc:
            best_acc = val_acc
            print(f"net {net_id} found a new best accuracy on validation set.\n validation set accuracy = {val_acc} ")
            print(f"hyperparameters: \n LR = {hyper_combo[0]} \n batch size = {hyper_combo[1]} \n reg = {hyper_combo[2]} \n num hidden = {hyper_combo[3]}")
        print(f"net {net_id} finished run {count} ")
        count+=1
    print("-------------------------------------------------")                        
    print(f"NET {net_id} BEST OVERALL ACC: {best_acc}\n")    
    print(f"hyperparameters: \n LR = {hyper_combo[0]} \n batch size = {hyper_combo[1]} \n reg = {hyper_combo[2]}")
    print("-------------------------------------------------")                        


In [ ]:
np.random.seed(0)
#passing random values into grid search function

lr = np.random.uniform(low=.05, high=.15, size=(3))
batch = np.random.uniform(low=300, high=600, size=(3)).astype("int")
reg = np.random.uniform(low=0.0001, high=0.0001, size=(3))
hidden = np.random.uniform(low=35, high=65, size=(3)).astype("int")

grid_search(lr, batch, reg, hidden)

In [ ]:
# net 2 found a new best accuracy on validation set.
#  validation set accuracy = 0.438 
# hyperparameters: 
#  LR = 0.12151893663724195 
#  batch size = 463 
#  reg = 0.0001 
#  num hidden = 46

### 3f. Plot STL-10 results with best hyperparameters

Train an MLP with the best hyperparameters that you found from your parameter search and create two plots:
- Training STL-10 loss curve
- Training and validation set STL-10 accuracy curves

In [ ]:
random.seed(0)
bestNet = MLP(3072, 20, 10)
loss_history, train_history, val_history = bestNet.fit(x_train, y_train, x_val, y_val, n_epochs= 200, verbose=1, print_every=100, lr=0.1215, mini_batch_sz=463, reg= .0001)

test_acc = bestNet.accuracy(y_test, bestNet.predict(x_test))
plot_cross_entropy_loss(loss_history)

plt.plot(val_history)
plt.plot(train_history)
plt.legend(["validation set accuracy","training set accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.show()
print(f"test acc is {test_acc}")

**Question 8**: Use your best trained network to compute the accuracy on the test set after 200 epochs. What accuracy do you get?

**Question 9:** Why would you use random search over grid search when optimizing parameters on a dataset?

**Answer 8**: The test set accuracy computed on the test set is ~38%.

**Answer 9:** A random search is better when some hyperparameters are not as important as others. Selecting randomly gives high variance but can occasionally result in significantly better performances by chance.

### 3g. Visualize learned weights

Run the `plot_weights` function to generate a grid visualization of them.

You should see structure in the weights if your network is performing well. If you have a large number of hidden units, some may not be "used" so a subset of the weights may resemble "noise".

In [ ]:
best_y_wts = bestNet.get_y_wts()
best_y_wts = best_y_wts.reshape(32, 32, 3, -1).transpose(3, 0, 1, 2)

In [ ]:
def plot_weights(wts, maxRows=25, verbose=0):
    # limit height of figure by number of neurons
    grid_sz = int(maxRows)
    grid_sz = np.minimum(grid_sz, int(np.sqrt(len(wts))))

    if verbose > 0:
        print(f'Showing {grid_sz} rows')
    
    plt.figure(figsize=(20,20))
    for x in range(grid_sz):
        for y in range(grid_sz):
            lin_ind = np.ravel_multi_index((x, y), dims=(grid_sz, grid_sz))
            plt.subplot(grid_sz, grid_sz, lin_ind+1)
            currImg = wts[lin_ind]
            low, high = np.min(currImg), np.max(currImg)
            currImg = 255*(currImg - low) / (high - low)
            currImg = currImg.astype('uint8')
            plt.imshow(currImg)
            plt.gca().axis('off')
    plt.show()

In [ ]:
plot_weights(best_y_wts, verbose=1)

## Extensions

**Reminder**: Please do not integrate extensions into your base project so that it changes the expected behavior of core functions. It is better to duplicate the base project and add features from there.

1) Analyze the differences between training when sampling with replacement (i.e. not every input sample is usually processed on an epoch) and sampling without replacement (e.g. time, accuracy, loss, etc).

2) Investigate how the single layer softmax network does with the CIS dataset. Explain and provide plots showing your results.

3) If you have time to spare (or want to throw more computing power at the STL-10 dataset), process through the SLP and MLP and tune hyperparameters with the dataset at its original resolution (96x96 images). Show images of your learned weights. Can you find a training sweet spot where the learned weight visualizations look particularly cool?

4) Implement a multi-class sigmoid classifer. I suggest creating another subclass of `SoftmaxLayer` and/or `MLP`. Compare and contrast results achieved by the softmax network.

5) Explore alternative MLP architectures and compare/constrast results and performance with the ones used in the base project. For example, replace hidden layer activation function with sigmoid, add one or more additional hidden layers, etc. 

6) Explore the effects of batch gradient descent, stochastic gradient descent, and mini-batch gradient descent. Make plots and interpret your results.

7) Obtain, preprocess, train, and evaluate the performance of `SoftmaxLayer` and/or `MLP` on another dataset with comparable types of image features. MNIST is a good one.

8) Make a fancy coarse-to-fine grid search that automatically "zooms in" on the best hyperparameter combination ranges several times.

**Extension: Multithreaded Grid Search and Random Search**

This extension is baked into the project notebooks, but not the base code. In order to do a more efficient grid/random search, I implemented a multiprocess solution with python's multiprocessing library. This library enables us to write functions that can be executed concurrently. Grid searching is especially well cut out for optimization via concurrency due to the results of each run of the net not directly depending on one another. We do of course have to eventually compare validation set accuracy across threads, but this can be done by the user by printing out the lifetime best accuracy of each thread as you go along. 

It took a few iterations to come up with this solution, but eventually, I created logic in a grid search function that takes in numpy linspaces with the desired number of parameters entered already. This way, the user can enter parameters in a familiar way. The grid search function creates a queue of jobs to be executed, (pretty much) evenly distributed over the appopriate number of threads. Then, the specified number of threads is initialized (I had luck with 5), and each thread works through its individual queue at its own pace, printing best accuracy values as it goes. The threads are joined at the end. The result is a grid search function that uses as significantly higher percentage of my CPU while running and works faster than my single threaded program by a factor of roughly 2 to 5 depending on the specific parameters. 

Adapting the grid search function for random searching was as simple as changing the expected linspaces to random uniform 1D arrays within given high and low ranges, as well as adding an additional parameter due to the number of hidden layers in an MLP.




In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

# for obtaining the STL-dataset
import load_stl10_dataset

# for preprocessing dataset
import preprocess_data

from softmax_layer import SoftmaxLayer

# Set the color style so that Professor Layton can see your plots
plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
# Make the font size larger
plt.rcParams.update({'font.size': 20})

# Turn off scientific notation when printing
np.set_printoptions(suppress=True, precision=3)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

random.seed(0)

def plot_cross_entropy_loss(loss_history):
    plt.plot(loss_history)
    plt.xlabel('Training iteration')
    plt.ylabel('loss (cross-entropy)')
    plt.show()


In [ ]:
# get CIS data from base project
val_size = 20

cis_train_path = os.path.join('data', 'cis', 'cis_train.dat')
cis_test_path = os.path.join('data', 'cis', 'cis_test.dat')

cis_train_all = np.loadtxt(cis_train_path, delimiter='\t')

s_inds = np.arange(len(cis_train_all))
np.random.seed(0)
np.random.shuffle(s_inds)

cis_train_all = cis_train_all[s_inds]

cis_train_x = cis_train_all[:, :2]
cis_train_y = cis_train_all[:, 2].astype(int)

cis_val_x = cis_train_x[:val_size]
cis_train_x = cis_train_x[val_size:]
cis_val_y = cis_train_y[:val_size]
cis_train_y = cis_train_y[val_size:]

cis_test_all = np.loadtxt(cis_test_path, delimiter='\t')
cis_test_x = cis_test_all[:, :2]
cis_test_y = cis_test_all[:, 2].astype(int)

softmax_cis = SoftmaxLayer(2)
loss_history = softmax_cis.fit(cis_train_x, cis_train_y, n_epochs=100, mini_batch_sz=20)

In [ ]:
# plot loss over time
plot_cross_entropy_loss(loss_history)

In [ ]:
# plot predictions
test_pred_sm = softmax_cis.predict(cis_test_x)
plt.scatter(cis_test_all[:,0], cis_test_all[:,1], c=test_pred_sm)
plt.axis('equal')
plt.title("Softmax Predicted CIS Test Data")
plt.show()

For this extension, we tested out the softmax single-layer network with the CIS dataset. The loss graph demonstrates that the loss does not decrease over time, and it fluctuates between increasing and decreasing. Additionally, we see that the softmax network does not predict any points as part of a circle, and so we were correct in our prediction that the softmax single-layer network would be an inapropriate choice for the CIS dataset. Since there is no hidden layer performing ReLU activation, the network simply sees the binary classification of the CIS data but does not understand how the individual points that are of the 'circle' class connect with one another to draw a circle in a square.

In [ ]:
from mlp import MLP
from sklearn.datasets import fetch_openml
iris_full=fetch_openml(data_id=61)
mnist_full = fetch_openml('mnist_784')

#MNIST import
mnist = mnist_full.data.astype(np.float64)
mnist_classes = mnist_full.target.astype(np.int)
mnist_dev = mnist[:1000]
mnist_dev_classes = mnist_classes[:1000]
x_train_mnist, y_train_mnist, x_test_mnist, y_test_mnist, x_val_mnist, y_val_mnist, x_dev_mnist, y_dev_mnist = preprocess_data.create_splits(mnist, mnist_classes, 40000, 20000, 5000, 5000)

#standardizing
mnist_dev_std = (mnist_dev- np.mean(mnist_dev))/np.std(mnist_dev)
mnist_dev_std = (mnist_dev_std - np.mean(mnist_dev_std,axis=0))/np.std(mnist_dev_std,axis=0)

softmax_mnist = SoftmaxLayer(10)
#print(x_train_mnist.shape)
#print(y_train_mnist.shape)
#print(mnist_classes)
loss_history_mnist = softmax_mnist.fit(x_train_mnist, y_train_mnist, n_epochs=50)

In [ ]:
# plot loss over time
plot_cross_entropy_loss(loss_history_mnist)
print("First loss:", loss_history_mnist[0])
print("First loss:", loss_history_mnist[-1])

For this extension, we tested out the softmax single-layer network with the MNIST dataset of digits The loss graph demonstrates that the loss did decrease over time from about 30 to 0.72. The decrease wasn't linear, which suggests that different hyperparameters might be used to acheive a less noisy loss graph.